In [1]:
pip install streamlit


     ---------------------------------------- 8.1/8.1 MB 22.5 MB/s eta 0:00:00
     --------------------------------------- 21.5/21.5 MB 15.6 MB/s eta 0:00:00
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached validators-0.20.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 4.7/4.7 MB 18.8 MB/s eta 0:00:00
     ------------------------------------- 471.5/471.5 kB 14.9 MB/s eta 0:00:00
     ------------------------------------- 422.5/422.5 kB 12.9 MB/s eta 0:00:00
     ---------------------------------------- 239.4/239.4 kB ? eta 0:00:00
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ------------------------------------- 188.5/188.5 kB 11.1 MB/s eta 0:00:00
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached Pygment

In [1]:
import warnings
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('cleaned_movies.csv')
df

,actor/actress,category,type,title,year,duration,genres,rating,votes
0,Fred Astaire,actor,movie,The Gay Divorcee,1934,107,"Comedy,Musical,Romance",7.4,8322
1,Ginger Rogers,actress,movie,The Gay Divorcee,1934,107,"Comedy,Musical,Romance",7.4,8322
2,Edward Everett Horton,actor,movie,The Gay Divorcee,1934,107,"Comedy,Musical,Romance",7.4,8322
3,Alice Brady,actress,movie,The Gay Divorcee,1934,107,"Comedy,Musical,Romance",7.4,8322
4,Fred Astaire,actor,movie,Roberta,1935,106,"Comedy,Musical,Romance",7.0,3560
...,...,...,...,...,...,...,...,...,...
6075698,Gaia Delepine,actress,short,Entre Deux Stations,2017,5,Short,8.9,13
6075699,Alejandro Bordier,actor,short,Entre Deux Stations,2017,5,Short,8.9,13
6075700,James Craigmyle,self,tvEpisode,Top 10 K9 Moments,2019,\N,"Crime,Reality-TV",7.5,9
6075701,James Craigmyle,self,tvEpisode,Raising the Woof,2020,41,Reality-TV,8.4,16


- Movie recommendation by actor/actress

In [3]:
def get_actor_movies(actor_name, n_movies=5):
    actor_movies = df[df['actor/actress'] == actor_name]
    if len(actor_movies) == 0:
        return None
    actor_movies = actor_movies.sample(min(len(actor_movies), n_movies))
    return actor_movies

- Movie recommender by genre

In [5]:
def get_similar_movies(movie_title, n_movies=5):
    if movie_title not in df['title'].values:
        return None
    movie_genre_str = df[df['title'] == movie_title]['genres'].values[0]
    movie_genres = movie_genre_str.split(',')
    
    same_genre_movies = df[df['genres'].apply(lambda x: any(genre in x.split(',') for genre in movie_genres))]
    same_genre_movies = same_genre_movies[same_genre_movies['title'] != movie_title]
    if len(same_genre_movies) == 0:
        return None
    similar_movies = same_genre_movies.sample(min(len(same_genre_movies), n_movies))
    return similar_movies

In [6]:
#Testing the two
get_actor_movies('Tom Hanks')

,actor/actress,category,type,title,year,duration,genres,rating,votes
104225,Tom Hanks,producer,movie,Parkland,2013,93,"Drama,History,Mystery",6.4,16926
104090,Tom Hanks,archive_footage,tvMovie,Tom Hanks: Hollywood's Mr Nice Guy,2022,67,"Biography,Documentary",5.8,30
104421,Tom Hanks,self,tvMiniSeries,The Eighties,2016,42,"Documentary,History",8.2,1432
99027,Tom Hanks,self,tvEpisode,Penny Marshall,2003,\N,"Biography,Documentary",9.7,215
103388,Tom Hanks,actor,movie,The Terminal,2004,128,"Comedy,Drama,Romance",7.4,475114


In [7]:
get_actor_movies('Julia Roberts')

,actor/actress,category,type,title,year,duration,genres,rating,votes
97911,Julia Roberts,actress,movie,Conspiracy Theory,1997,135,"Action,Mystery,Thriller",6.7,104195
143627,Julia Roberts,actress,short,Happy Diamonds,2021,1,Short,6.4,74
54988,Julia Roberts,self,tvEpisode,George Clooney/Julia Roberts/Omar Apollo,2022,\N,"Comedy,Music,Talk-Show",8.2,17
143516,Julia Roberts,actress,movie,Erin Brockovich,2000,131,"Biography,Drama",7.4,208998
143785,Julia Roberts,self,tvSeries,Busy Tonight,2018,22,Talk-Show,6.2,328


In [8]:
#I will filter to only get movies
def get_actor_movies(actor_name, n_movies=5):
    actor_movies = df[(df['actor/actress'] == actor_name) & (df['type'] == 'movie')]
    if len(actor_movies) == 0:
        return None
    actor_movies = actor_movies.sample(min(len(actor_movies), n_movies))
    return actor_movies

In [9]:
def get_similar_movies(movie_title, n_movies=5):
    if movie_title not in df['title'].values:
        return None
    movie_genre_str = df[df['title'] == movie_title]['genres'].values[0]
    movie_genres = movie_genre_str.split(',')
    
    same_genre_movies = df[df['genres'].apply(lambda x: any(genre in x.split(',') for genre in movie_genres)) & (df['type'] == 'movie')]
    same_genre_movies = same_genre_movies[same_genre_movies['title'] != movie_title]
    if len(same_genre_movies) == 0:
        return None
    similar_movies = same_genre_movies.sample(min(len(same_genre_movies), n_movies))
    return similar_movies

In [10]:
get_actor_movies('Meryl Streep')

,actor/actress,category,type,title,year,duration,genres,rating,votes
299132,Meryl Streep,actress,movie,A Prairie Home Companion,2006,105,"Comedy,Drama,Music",6.7,23272
430324,Meryl Streep,actress,movie,Disneynature: Wings of Life,2011,81,"Documentary,Family,News",7.3,679
215485,Meryl Streep,actress,movie,Suffragette,2015,106,"Drama,History",6.9,43563
104550,Meryl Streep,actress,movie,The Post,2017,116,"Biography,Drama,History",7.2,157153
430391,Meryl Streep,actress,movie,Ginevra's Story: Solving the Mysteries of Leon...,1999,55,"Biography,Documentary",4.5,68


In [11]:
get_actor_movies('Halle Berry')

,actor/actress,category,type,title,year,duration,genres,rating,votes
103931,Halle Berry,actress,movie,Cloud Atlas,2012,172,"Drama,Mystery,Sci-Fi",7.4,368457
544468,Halle Berry,actress,movie,Kidnap,2017,95,"Action,Crime,Thriller",5.9,38264
526954,Halle Berry,self,movie,Sidney,2022,111,"Biography,Documentary",7.8,1048
160428,Halle Berry,actress,movie,Swordfish,2001,99,"Action,Crime,Thriller",6.5,192347
132759,Halle Berry,actress,movie,Robots,2005,91,"Adventure,Animation,Comedy",6.4,155405


In [12]:
get_similar_movies('Robots')

,actor/actress,category,type,title,year,duration,genres,rating,votes
3172129,Pascal N'Zonzi,actor,movie,Premier de la classe,2019,82,Comedy,5.3,188
1434209,Lorena Velázquez,actress,movie,Un gallo con espolones (Operación ñongos),1964,114,"Comedy,Family,Musical",5.9,11
156286,Paul Dawson,actor,movie,The Big Kahuna,1999,90,"Comedy,Drama",6.6,14799
4068440,Anthony Karoui,actor,movie,Peau de cochon,2004,84,"Comedy,Documentary",6.8,83
811199,John Fusco,writer,movie,Hidalgo,2004,136,"Action,Adventure,Biography",6.7,84089


In [13]:
get_similar_movies('Jurassic Park')

,actor/actress,category,type,title,year,duration,genres,rating,votes
2261096,Ken Hare,actor,movie,Six Soldiers,1974,94,"Action,Adventure,Drama",5.8,19
5843244,Aktion,actor,movie,The Shop,2014,93,"Action,Drama",5.0,61
5060230,Barkha Madan,actress,movie,Surkhaab,2014,100,"Adventure,Drama,Thriller",6.5,80
5557258,Chloe Blue,cinematographer,movie,Guns and Grams,2016,105,"Action,Crime,Drama",8.5,45
1808820,Reno Wilson,actor,movie,Tooken,2015,80,"Action,Comedy",3.7,1739


In [15]:
get_similar_movies('Superbad')

,actor/actress,category,type,title,year,duration,genres,rating,votes
438855,Stuart Cornfeld,producer,movie,Dodgeball: A True Underdog Story,2004,92,"Comedy,Sport",6.7,259194
4670601,Sakura Ando,actress,movie,Love Exposure,2008,237,"Action,Comedy,Drama",8.0,15011
1441900,Samira Ahmed,actress,movie,The Circus,1968,95,"Comedy,Drama,Music",5.5,37
5677172,Ella Cruz,actress,movie,Steal,2021,98,Comedy,6.1,147
2231467,Sener Sen,actor,movie,Abbas in Flower,1982,87,"Comedy,Drama,Romance",8.0,10856
